# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
#import csv of Cities and weather data
weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_df = weather_df.dropna()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#configure and plot on gmaps 
gmaps.configure(api_key=gkey)
fig=gmaps.figure(zoom_level=2, center=(0,0))
locations = weather_df[['Lat','Lng']]
heat_layer = gmaps.heatmap_layer(locations, weights=weather_df['Humidity'], max_intensity=100)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
#narrow dataframe to cities of desired weather conditions
narrow_temp = weather_df.loc[(weather_df['Max Temp']>75) & (weather_df['Max Temp']<85)]
narrow_wind = narrow_temp.loc[weather_df['Wind Speed']<10]
hotel_df = narrow_wind.loc[narrow_wind['Cloudiness'] < 10]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
53,takoradi,2,GH,1579889596,73,4.88,-1.76,82.83,8.32
83,kavieng,0,PG,1579889601,65,-2.57,150.80,83.28,5.97
154,sumenep,3,ID,1579889500,69,-7.02,113.87,82.38,1.01
249,lorengau,5,PG,1579889628,70,-2.02,147.27,83.53,5.53
328,chuy,0,UY,1579889642,44,-33.70,-53.46,82.09,6.60
427,port hedland,0,AU,1579889449,88,-20.32,118.57,82.40,6.93
436,les cayes,0,HT,1579889660,59,18.20,-73.75,78.37,0.31
502,marawi,8,PH,1579889671,90,8.00,124.29,77.00,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
#create empty list to append to
hotel_df['Hotel Name'] = ""
hotels = []
hotel_lat = []
hotel_lng = []
hotel_city = []
hotel_county = []

#set parameters
target_coordinates = hotel_df[['Lat', 'Lng']]
target_search = "Hotel"
rankby= 'distance'
target_type = "lodging"

params = {
     "location": "",
    "keyword": target_search,
    "rankby": rankby,
    "type": target_type,
    "key": gkey
}

#base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, location in target_coordinates.iterrows():
    
    params['location'] = f"{location[0]},{location[1]}"     #add current lat and long to params
    response = requests.get(base_url, params=params).json()
    
    #append list 
    hotel_lat.append(response['results'][0]['geometry']['location']['lat'])
    hotel_lng.append(response['results'][0]['geometry']['location']['lng'])
    hotel_city.append(response['results'][0]['vicinity'])
    hotels.append(response['results'][0]['name'])

hotel = pd.DataFrame({
    'Hotel Name':hotels,
    'City':hotel_city,
    'Lat':hotel_lat,
    'Lng':hotel_lng    
})


C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
#create list of hotel locations for mapping(I did this before I realized I needed lat and long within the dataframe itself)
hotel_loc = [[hotel_lat[i], hotel_lng[i]] for i in range(len(hotel_lat))]
hotel_loc
hotel_df['Hotel Name'] = hotels
hotel

C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Hotel Name,City,Lat,Lng
0,Planters Lodge and Spa,"Dixcove Road, Takoradi",4.882547,-1.757904
1,Kavieng Niu Lodge,"Tanga St, Kavieng",-2.570513,150.795923
2,Adhirasa House,"Jl. Raya Adirasa No.09, Kothe, Kolor, Kabupate...",-7.020220,113.866357
3,Seeadler Bay Hotel,"Lot 31 Section 31, Lorengau 641",-2.019335,147.267315
4,Gaboto,"Gaboto 918, Chuy",-33.699793,-53.464089
5,The Esplanade Hotel,"2 Anderson St, Port Hedland",-20.314012,118.576470
6,Caribbean Hôtel Cayes,"Rue Union, Et Rue Auxilius Fougere, Les Cayes",18.196272,-73.749804
7,Marawi Resort Hotel,Marawi City,7.997245,124.264040


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]


In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_loc, label=hotels, info_box_content=hotel_info, display_info_box=True)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))